In [1]:
#Run this cell if you do not have tensorflow, opencv, scikit-learn, matplotlib, or mediapipe installed
!pip install tensorflow==2.12.0 opencv-python scikit-learn matplotlib mediapipe

In [2]:
### import various necessary libraries ###
from matplotlib import pyplot as plt
import numpy as np
import mediapipe as mp
import cv2
import os
import time

In [3]:
### initialize holistic and drawing mediapipe variables ###
mpHolistic = mp.solutions.holistic
mpDrawing = mp.solutions.drawing_utils

In [4]:
### initialize function to detect structures using mediapipe ###
def mp_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Write colors to RGB
    image.flags.writeable = False                  # Image unwritable
    results = model.process(image)                 # Process image
    image.flags.writeable = True                   # Image writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Convert colors back to BGR
    return image, results

In [5]:
### initialize function responsible for drawing mediapipe connections ###
def draw_landmarks(image, results):
    mpDrawing.draw_landmarks(image, results.pose_landmarks, mpHolistic.POSE_CONNECTIONS) # Draw pose connections
    mpDrawing.draw_landmarks(image, results.left_hand_landmarks, mpHolistic.HAND_CONNECTIONS) # Draw left hand connections
    mpDrawing.draw_landmarks(image, results.right_hand_landmarks, mpHolistic.HAND_CONNECTIONS) # Draw right hand connection

In [6]:
### function used to add styles to landmarks ###
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mpDrawing.draw_landmarks(image, results.pose_landmarks, mpHolistic.POSE_CONNECTIONS,
                             mpDrawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mpDrawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mpDrawing.draw_landmarks(image, results.left_hand_landmarks, mpHolistic.HAND_CONNECTIONS, 
                             mpDrawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mpDrawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mpDrawing.draw_landmarks(image, results.right_hand_landmarks, mpHolistic.HAND_CONNECTIONS, 
                             mpDrawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mpDrawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
### initialize function that extracts keypoints for landmarks ###
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [8]:
### initialize actions list with all phrases used in the model ###
actions = np.array(['hello', 'goodbye', 'thank you', 'how', 'are you', 'take care'])

In [9]:
### import necessary tensorflow functions ###
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Flatten
from tensorflow.keras.callbacks import TensorBoard

In [10]:
### initialize model with proper trained sequence ###
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(10,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [11]:
### load in pre-trained model ###
model.load_weights('action.h5')

In [12]:
### import stats library ###
from scipy import stats

In [13]:
### test model in real time ###
sequence = []
sentence = ""
predictions = []
threshold = 0.99

cap = cv2.VideoCapture(0)
# Set model 
with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mp_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-10:]
        
        if len(sequence) == 10:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        # Display word on screen
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    sentence = actions[np.argmax(res)]
                    
                else: 
                    sentence = ""

            
        cv2.rectangle(image, (0,0), (640, 40), (0, 0, 0), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 1s 1s/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 78ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 48ms/step
<class 'mediapipe.python.solution_base.SolutionOutput

1/1 [==============================] - 0s 63ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 59ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 54ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 162ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 88ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 59ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 53ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 46ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [=============

1/1 [==============================] - 0s 40ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 40ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 39ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 39ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 49ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============